In [2]:
from modules.dynamic_memory_network import DynamicMemoryNetwork
import tensorflow as tf
import os
import numpy as np
from utils import *
from sklearn.model_selection import train_test_split

In [3]:
trainset = sklearn.datasets.load_files(
    container_path = 'data', encoding = 'UTF-8'
)
trainset.data, trainset.target = separate_dataset(trainset, 1.0)
print(trainset.target_names)
print(len(trainset.data))
print(len(trainset.target))

['negative', 'positive']
10662
10662


In [4]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [413, 10158, 77, 10159, 1641, 1061, 134, 68, 13, 96] ['michael', 'gerbosis', 'script', 'economically', 'packed', 'telling', 'scenes', 'first', 'time', 'years']


In [5]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [6]:
size_layer = 128
num_layers = 2
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128
decay_step = 1e4
decay_rate = 1.0
story_len = 1

In [7]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

model = DynamicMemoryNetwork(
    dimension_output,
    learning_rate,
    decay_step,
    decay_rate,
    maxlen,
    story_len,
    len(dictionary),
    embedded_size,
    size_layer
)

sess.run(tf.global_variables_initializer())

In [8]:
vectors = str_idx(trainset.data, dictionary, maxlen)
train_X, test_X, train_Y, test_Y = train_test_split(
    vectors, trainset.target, test_size = 0.2
)

In [9]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = train_X[i : min(i + batch_size, train_X.shape[0])]
        batch_y = train_Y[i : min(i + batch_size, train_X.shape[0])]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.answer_single: batch_y,
                model.query: batch_x,
                model.story: batch_x_expand,
                model.dropout_keep_prob: 1.0
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
        batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.answer_single: batch_y,
                model.query: batch_x,
                model.story: batch_x_expand,
                model.dropout_keep_prob: 1.0
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1


train minibatch loop:   3%|▎         | 2/67 [00:00<00:03, 17.43it/s, accuracy=0.695, cost=1.08]

epoch: 0, pass acc: 0.000000, current acc: 0.719120
time taken: 4.981959581375122
epoch: 0, training loss: 1.629185, training acc: 0.585404, valid loss: 1.180462, valid acc: 0.719120



train minibatch loop:   3%|▎         | 2/67 [00:00<00:03, 18.45it/s, accuracy=0.906, cost=0.614]

epoch: 1, pass acc: 0.719120, current acc: 0.755203
time taken: 4.176710367202759
epoch: 1, training loss: 0.862142, training acc: 0.820889, valid loss: 0.983111, valid acc: 0.755203



train minibatch loop:   3%|▎         | 2/67 [00:00<00:03, 16.93it/s, accuracy=0.969, cost=0.386]

...

time taken: 3.9544994831085205
epoch: 4, training loss: 0.261083, training acc: 0.994372, valid loss: 1.671401, valid acc: 0.720510



train minibatch loop:   3%|▎         | 2/67 [00:00<00:03, 18.17it/s, accuracy=0.992, cost=0.208]

time taken: 4.248157024383545
epoch: 5, training loss: 0.222305, training acc: 0.997538, valid loss: 2.069268, valid acc: 0.708563



test minibatch loop: 100%|██████████| 17/17 [00:00<00:00, 37.25it/s, accuracy=0.694, cost=1.88]

time taken: 4.133220672607422
epoch: 6, training loss: 0.209037, training acc: 0.992965, valid loss: 1.616928, valid acc: 0.727542

break epoch:7



In [10]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = test_X[i : min(i + batch_size, test_X.shape[0])]
    batch_y = test_Y[i : min(i + batch_size, test_X.shape[0])]
    batch_x_expand = np.expand_dims(batch_x,axis = 1)
    predict_Y += np.argmax(
        sess.run(
            model.logits,
            feed_dict = {
                model.answer_single: batch_y,
                model.query: batch_x,
                model.story: batch_x_expand,
                model.dropout_keep_prob: 1.0
            },
        ),
        1,
    ).tolist()
    real_Y += batch_y


validation minibatch loop: 100%|██████████| 17/17 [00:00<00:00, 34.06it/s]


In [11]:
from sklearn import metrics
print(metrics.classification_report(real_Y, predict_Y, target_names = ['negative','positive']))

              precision    recall  f1-score   support

    negative       0.71      0.72      0.72      1072
    positive       0.71      0.71      0.71      1061

    accuracy                           0.71      2133
   macro avg       0.71      0.71      0.71      2133
weighted avg       0.71      0.71      0.71      2133

